# Project 1: A/B Testing

Ecommerce business offer a group of whitelist customers a special promotion for the membership subscription, and they have 2 type of screens: (an original price of package is 199k)

Screen A: show a discounted price of paid package (99k)
Screen B: show a discount amount in price (discount 100k)

Tasks to be done:
1.	Design Ab testing experiment for this case. What can be the external factors in this AB testing? How can we monitor these factors?
2.	The data was collected as an attached file.
Assuming that the ab testing experiment was conducted in the perfect circumstances, use hypothesis testing to prove the results is significant


Dataset:

customer_id: unique id of each users

group: group1: show screenA | group2: show screenB

is_buy: whether that user buy the subscription or not

## A/B Experiment Design

### Define goal
The goal of this A/B test is to determine which screen design - Screen A (discounted price) or Screen B (discount amount) - is more effective at driving conversions for the whitelist customers on the membership subscription offering.

### Define variables to test
- Independent Variable: Screen design (Screen A vs. Screen B)

- Dependent Variable: Conversion rate on the membership subscription purchase

### Create Variants
- Variant A: Screen showing the discounted price of 99k

- Variant B: Screen showing the discount amount of 100k

### Split Audience
Randomly split the whitelist customer audience into two equal groups:

- Group 1 will be shown Variant A (discounted price)

- Group 2 will be shown Variant B (discount amount)

### External Impacts
#### Competitor Promotions:
If competitors in the ecommerce space are running their own promotions or discounts during the test period, it could influence the whitelist customers' purchasing decisions. This could make one screen design perform better than the other, not necessarily due to the screen design itself.

To mitigate: Carefully monitor the test environment and track any changes in the market or competitive landscape

To monitor:
- Regularly monitor competitor websites and marketing channels for any promotional activities related to similar membership subscriptions.
- Set up alerts or run weekly/monthly competitive analysis reports to stay informed.


#### Marketing campaigns
Any concurrent marketing campaigns, emails, or other promotional activities could interact with the screen designs and impact the test results.

To mitigate: Ensure there are no other major marketing or promotional activities happening concurrently.

To monitor:
- Closely track all marketing activities (emails, social posts, etc.) happening during the A/B test period.
- Maintain a calendar of marketing campaigns and promotions to identify any potential confounding factors.
- Analyze marketing attribution data to understand the impact of concurrent campaigns on the test results.

#### Seasonality
The time of year or season could impact customer purchasing behavior and willingness to subscribe, independent of the screen design.

To mitigate: Consider running the test over a longer duration to account for seasonal fluctuations

To monitor:
- Track historical sales data and conversion rates for the membership subscription over the past 12-24 months to identify seasonal trends.
- Double check the seasonaility index to estimate the expected performance of the to-do test.


### Conducting the Test
To see if Screen A would convert better than Screen B, I would use a one-sided Chi-Square test:

- Null Hypothesis (H0): The conversion rate for Screen A is less than or equal to the conversion rate for Screen B.

- Alternative Hypothesis (H1): The conversion rate for Screen A is greater than the conversion rate for Screen B.


In [ ]:
import pandas as pd

In [ ]:
df=pd.read_csv('abtesting-dataset.csv')

In [ ]:
df.head()

,Unnamed: 0,customer_id,group,is_buy
0,0,21393799,2,0
1,1,20349278,2,0
2,2,11655915,2,0
3,3,18516581,2,0
4,4,7805171,2,0


In [ ]:
from scipy.stats import chi2_contingency

In [ ]:
from scipy.stats import chi2

In [ ]:
import numpy as np

In [ ]:
# 2. Create the contingency table
group1 = df[df['group'] == 1] # Screen A group
group2 = df[df['group'] == 2] # Screen B group

In [ ]:
group1_converted = group1.loc[group1['is_buy'] == 1]['is_buy'].count()
group1_non_converted = group1.loc[group1['is_buy'] == 0]['is_buy'].count()

group2_converted = group2.loc[group2['is_buy'] == 1]['is_buy'].count()
group2_non_converted = group2.loc[group2['is_buy'] == 0]['is_buy'].count()

In [ ]:
contingency_table = np.array([[group1_converted, group1_non_converted],
                             [group2_converted, group2_non_converted]])

In [ ]:
contingency_table

[[ 5199  9826]
 [ 4357 10667]]


In [ ]:
# 3. Apply the Chi-Square test
result = chi2.cdf(contingency_table[0, 0], 1)  # 1 degree of freedom
p_value = 1 - result  # One-sided p-value

In [ ]:
# 4. Interpret the results
print(f"One-Sided Chi-Square Statistic: {contingency_table[0, 0]:.2f}")
print(f"p-value: {p_value:.4f}")

One-Sided Chi-Square Statistic: 5199.00
p-value: 0.0000


In [ ]:
# 5. Determine statistical significance
significance_level = 0.05
if p_value < significance_level:
    print("The conversion rate for Screen A is greater than the conversion rate for Screen B.")
else:
    print("The conversion rate for Screen A is not greater than the conversion rate for Screen B.")

The conversion rate for Screen A is greater than the conversion rate for Screen B.


### Insights

The conversion rate for Screen A is significantly greater than the conversion rate for Screen B.

The data provides overwhelming evidence (p-value: 0.0000) to reject the null hypothesis (H0) and support the alternative hypothesis (H1).

The difference in conversion rates between the two screens is highly statistically significant (One-Sided Chi-Square Statistic: 5199.00) and unlikely to have occurred by chance.

Therefore, I would advise using screen A while keeping these considerations in mind:
#### Consideration #1: Be aware of Practical Significance
Now, it's time to be considerate. I'm going to calculate the effect size to quantify the magnitude of the difference between the 2 conversion rates, telling me how large the actual difference is, beyond the statistical significance.

In [ ]:
import scipy.stats as stats

In [ ]:
# Calculate conversion rate
screen_a_converted = contingency_table[0, 0] / (contingency_table[0, 0] + contingency_table[0, 1])
screen_b_converted = contingency_table[1, 0] / (contingency_table[1, 0] + contingency_table[1, 1])

print(f"Conversion rate for screen A: {screen_a_converted:.2%}")
print(f"Conversion rate for screen B: {screen_b_converted:.2%}")

Conversion rate for screen A: 34.60%
Conversion rate for screen B: 29.00%


Effect size: 34.60% - 29.00% = 5.61%
The 5.61 percentage point difference may or may not be practically significant, depending on the specific context and goals of the analysis. To be more specific, if the overall conversion rates are low, a 5.61 percentage point difference could be quite meaningful. But if the overall conversion rates are already high, a 5.61 percentage point difference may not translate to a significant real-world impact. In this case, from 29%-34.60% could be translated into being meaningful.

#### Consideration #2: Be aware of Downstream Metrics

1. Identify potential downstream metrics:

Think about what other important business metrics or KPIs could be impacted by the choice of screen design (screen A vs. screen B).

Some examples could include:
- Total revenue or sales
- Customer lifetime value
- Repeat purchase rate
- Customer satisfaction
- User engagement metrics (e.g., time on site, pages viewed)

2. Analyze the downstream impact:
- Examine how the choice of screen design (A vs. B) affects these downstream metrics, not just the immediate conversion rate.
- For example, if screen A has a higher conversion rate but leads to lower customer satisfaction or repeat purchases, the overall business impact may not be as positive as the conversion rate alone suggests.
- Look for any unexpected effects on the downstream metrics.

3. Evaluate the full picture:
- Don't rely solely on the conversion rate difference as the measure of success. Take a holistic view and consider the impact on the broader set of relevant metrics.
- The goal should be to understand how the choice of screen design affects the entire customer journey and business performance, not just the initial conversion.

4. Iterate
- Use the insights from the downstream metric analysis to inform future iterations of the screen design or other product/marketing decisions.
- Continuously monitor the key downstream metrics to ensure the chosen design is delivering the desired overall business impact.

#### Consideration #3:  Be aware of the local maximum

Consider the test duration, which should not linger more than expected as there might be better options beyond screen A and B. Besides, there would be more campaigns coming.